# Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [2]:
# imports
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime

In [10]:
def scrap_habr(search_str, page=1):
    """
    :param
      search_str: поисковый запрос array
      page: пагинация

    :return DF Pandas
    """

    scrap_df = pd.DataFrame()
    params_req = {
        'url': 'https://habr.com/ru/search/',
        'search': {
            'target_type': 'posts',
            'order': 'relevance',
        }}
    for search in search_str:
        for i in range(1,page+1):
            print(f'Ищем {search} на странице', i)
            params_req['search']['q'] = search
            try:
                resp = req.get(params_req['url'] + f'page{i}/', params=params_req['search'])
                to_data = bs(resp.text)
                data = to_data.find_all('article', class_='tm-articles-list__item')
                for item in data:
                    date = datetime.strptime(item.find('time')['title'],'%Y-%m-%d, %H:%M')
                    title = item.find('h2', class_='tm-article-snippet__title').text
                    link = 'https://habr.com' + item.find('a', class_='tm-article-snippet__title-link').get('href')
                    likes = item.find('span', class_='tm-votes-meter__value_rating').text
                    try:
                        req_post = req.get(link)
                        full_text = bs(req_post.text).find('div', class_='article-formatted-body').text.strip()
                        row = {
                            'date': date,
                            'title': title,
                            'link': link,
                            'likes': likes,
                            'full_text': full_text,
                        }
                        scrap_df = pd.concat([scrap_df, pd.DataFrame([row])])
                        time.sleep(0.1)
                        print(f'parse post "{title}" done')
                    except NameError:
                        print(f'Не удалось распарсить статью: {title} (link: {link})')
                        print(NameError)
                time.sleep(0.2)
            except NameError:
                print('Something wrong', NameError)
            print(f'page {i} scrap done')
    return scrap_df.reset_index(drop=True)

df = scrap_habr(['react', 'nest'], 1)
df


Ищем react на странице 1
parse post "Яндекс.Практикум запустил курс «React-разработчик»" done
parse post "Энтузиаст создает интерфейс Windows 11 на React в браузере" done
parse post "Процветание проектов, React-router v6, RTK query и карьерная лестница в ад на Meet Up JS" done
parse post "React.js — Руководство для Rails разработчиков" done
parse post "Введение в React" done
parse post "Что такое React Native? Комплексное руководство 2021" done
parse post "Обзор релиз-кандидата React v0.14" done
parse post "Создаем приложение на JavaScript с помощью React Native" done
parse post "Вышел React v16.0" done
parse post "Новшества серверного рендеринга в React 16" done
parse post "Анализ и оптимизация React-приложений" done
parse post "Записки фрилансера: разработка первого React Native-приложения" done
parse post "Импорт react с древнейших времен до наших дней" done
parse post "Книга «React: современные шаблоны для разработки приложений 2-е издание»" done
parse post "React 18" done
parse po

,date,title,link,likes,full_text
0,2021-02-26 12:32:00,Яндекс.Практикум запустил курс «React-разработ...,https://habr.com/ru/company/yandex_praktikum/n...,+6,Сервис онлайн-образования Яндекс.Практикум зап...
1,2021-08-27 16:10:00,Энтузиаст создает интерфейс Windows 11 на Reac...,https://habr.com/ru/news/t/575002/,+11,Разработчик под ником Blue Edge опубликовал на...
2,2022-05-24 10:14:00,"Процветание проектов, React-router v6, RTK que...",https://habr.com/ru/company/alfa/news/t/667470/,+13,В среду 1 июня проведём гибридный митап с кома...
3,2016-04-21 23:16:00,React.js — Руководство для Rails разработчиков,https://habr.com/ru/post/281735/,+7,На начальном уровне такие переводы — мой вклад...
4,2020-03-22 16:19:00,Введение в React,https://habr.com/ru/post/461541/,+4,Оглавление\n1. Getting started with React\r\n....
5,2021-12-17 08:00:00,Что такое React Native? Комплексное руководств...,https://habr.com/ru/post/596183/,0,Вот уже несколько лет React Native является го...
6,2015-09-12 01:19:00,Обзор релиз-кандидата React v0.14,https://habr.com/ru/post/266683/,+16,Мы рады представить вам наш первый релиз-канди...
7,2016-06-21 17:43:00,Создаем приложение на JavaScript с помощью Rea...,https://habr.com/ru/company/plarium/blog/303328/,+27,В этом уроке мы будем изучать React Native – ф...
8,2017-09-28 16:42:00,Вышел React v16.0,https://habr.com/ru/post/338932/,+22,Это перевод поста Эндрю Кларка о выходе столь ...
9,2017-10-02 15:21:00,Новшества серверного рендеринга в React 16,https://habr.com/ru/company/ruvds/blog/339148/,+33,"Вышел React 16! Рассказывая об этом событии, м..."
